# # Aula \#5 - Desafio em Grupo & Checkpoint \#1 - Orientação a objetos e dicas

In [ ]:
import numpy as np
import pandas as pd

from utils.hints import Helper

Ao longo dos desafios, você foi usando dicas? Não?

Vamos ver como foi seu uso?

In [ ]:
helper = Helper()
helper.get_usage()

## Exercício

A versão atual da classe `Helper` somente mostra o número absoluto de exercícios para os quais foram pedidas dicas. Que tal incluir também quanto isso representa do total de dicas? Poderia ser algo como:

* `Você usou dicas em 2 exercícios. Isso representa 25% dos exercícios.`

Note que a quantidade de exercícios com dicas é a quantidade de arquivos de extensão `.pkl` na pasta `utils/.data`.

Carregue o código de `utils/hints.py` para que possamos modificar o código do método `get_usage` da classe `Helper` aqui.

In [ ]:
%load utils/hints.py

Quando terminar, execute novamente e veja se funcionou :)

In [ ]:
helper = Helper()
helper.get_usage()

Crie um gerador de dataframes aleatórios. A ideia é criar uma classe `RandomDataFrameGenerator` que em sua inicialização deverá receber um dicionário em que a chave é o tipo dos elementos (em string) e o valor é uma lista com os nomes da coluna daquele tipo.

Para nosso uso, somente será necessário aceitar três tipos apenas: `'int'`, `'float'` e `'str'`. Caso o tipo não pertença a nenhuma dessas categorias, deve-se exibir uma mensagem na inicialização, avisando ao usuário que coluna(s) não será(ão) criada(s) por não pertencer aos tipos aceitos.

Lembre-se de criar também o método `get_df` que retornará o dataframe criado. O método `get_df` deverá receber como parâmetro `n`, que será o tamanho do dataframe.

In [ ]:
class RandomDataFrameGenerator(object):
    def __init__(self, type2colname):
        ###
    
    def get_df(self, n):
        ###

In [ ]:
df_generator = RandomDataFrameGenerator({'str': ['col_str1', 'col_str2'], 'int': ['col_int'], 'float': ['col_float'], 'bla': ['col_bla']})

**Testes de sanidade**

Imprima os tipos e veja se faz sentido:

* as colunas que são de `string` devem ter dtype `object`
* as colunas que são de `int` devem ter dtype `int64`
* as colunas que são de `float` devem ter dtype `float64`

In [ ]:
df_generator.get_df(3).dtypes

In [ ]:
len(df_generator.get_df(3)) == 3

In [ ]:
len(df_generator.get_df(0)) == 0

## Dicas gerais

### tqdm

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
very_big_list = np.random.random(1000000)

In [ ]:
%%time
double_very_big_list = []
for elem in tqdm_notebook(very_big_list):
    double_very_big_list.append(2*elem)

### list/dict comprehensions



Veja que podemos também calcular `double_very_big_list` usando uma `list comprehension`:

In [ ]:
%%time
double_very_big_list = [2*elem for elem in tqdm_notebook(very_big_list)]

**Obs.:** Apesar de facilitar a escrita do código, as `list comprehensions` ou `dict comprehensions` devem ser usadas com moderação, afinal, queremos escrever um código que seja fácil para um humano ler.

### numpy

In [ ]:
%%time
np_double_very_big_list = 2*very_big_list

Conferindo se era tudo igual mesmo...

In [ ]:
np.all(double_very_big_list == np_double_very_big_list)

### pandas

**Leitura**

Usar `nrows` para checar que tipo de colunas existe no dataframe, se há ou não header, etc.

In [ ]:
%%time
df = pd.read_csv('data/datasets/winemag-data-130k-v2.csv', nrows=10)

In [ ]:
df.head(n=1)

Sem esse parâmetro, o dataset demora muito mais para carregar...

In [ ]:
%%time
df = pd.read_csv('data/datasets/winemag-data-130k-v2.csv')

Para ler arquivos muito grandes, o ideal é usar `chunksize` para ler o dataset aos poucos.

Ao usar o `chunksize`, ao invés de receber como retorno um objeto `pandas.DataFrame`, o que é retornado é um iterador de dataframes. Para reconstruir o dataframe total, basta percorrer esse iterador e ir concatenando cada pequeno dataframe.

In [ ]:
%%time
df = pd.DataFrame()
chunks = pd.read_csv('data/datasets/winemag-data-130k-v2.csv', chunksize=10000)
for little_df in tqdm_notebook(chunks):
    df = pd.concat([df, little_df])

**apply**

Uma das ferramentas bastante usadas para transformar colunas é o `apply`. Uma situação comum é querer usar o `apply` com uma função que precisa de outros parâmetros além das colunas do dataframe.

In [ ]:
example_df = df_generator.get_df(4)

Imagine que queremos aplicar a seguinte função à coluna `col_str2` para construir uma coluna `col_str3`.

In [ ]:
def format_str(text, fmt='**{}**'):
    return fmt.format(text)

In [ ]:
example_df['col_str3'] = example_df['col_str2'].apply(format_str, args=['##{}##'])

In [ ]:
example_df[['col_str2', 'col_str3']]

In [ ]:
example_df['col_str3'] = example_df['col_str2'].apply(format_str, fmt='~~{}~~')

In [ ]:
example_df[['col_str2', 'col_str3']]

Outra maneira de fazer isso, é usar o conceito de `nested functions` (funções aninhadas):

In [ ]:
def format_str_v2(fmt='**{}**'):
    def _format_str(text):
        return fmt.format(text)
    return _format_str

In [ ]:
example_df['col_str3'] = example_df['col_str2'].apply(format_str_v2(fmt='^^{}^^'))

In [ ]:
example_df[['col_str2', 'col_str3']]

Imagine agora que temos um dataframe muito grande...

In [ ]:
very_big_df = df_generator.get_df(1000000)

In [ ]:
%timeit very_big_df['double_col_int'] = very_big_df['col_int'].apply(lambda num: 2*num)

In [ ]:
%timeit very_big_df['double_col_int_v2'] = 2*very_big_df['col_int']

**apply + swifter**

[swifter](https://github.com/jmcarpenter2/swifter) é uma biblioteca que serve para aplicar funções a dataframes. A ideia é que ele vai otimizar o tempo de processamento, por procurar a melhor maneira de aplicar sua função (de forma paralela, de forma vetorizada etc.). Por enquanto, ela funciona bem para operações numéricas.

In [ ]:
import swifter

In [ ]:
%timeit very_big_df['double_col_int_v3'] = very_big_df['col_int'].swifter.apply(lambda num: 2*num)